In [1]:
import torch
import pickle
import os
from PIL import Image

## General Setup

In [2]:
import platform
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [3]:
def set_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu') 
    print(f"Using device: {device}")
    return device

device = set_device()

Using device: mps


## Fixed Models Setup

### Latent Codes

In [4]:
latents = torch.load(f"{DATA_PATH}/Models/e4e/experiments_default_lr/inversions/latents.pt").to(device)
file_paths = pickle.load(open(f"{DATA_PATH}/Models/e4e/experiments_default_lr/inversions/file_paths.pkl", 'rb'))
latents.shape

torch.Size([14060, 1, 16, 512])

### Generator Model

In [5]:
os.chdir(f"{ROOT_PATH}/stylegan2-ada-pytorch")
# Load model architecture
experiment_path = f"{DATA_PATH}/Models/Stylegan2_Ada/Experiments/00003-stylegan2_ada_images-mirror-auto2-kimg1000-resumeffhq512/"
model_name = "network-snapshot-000920.pkl"
model_path = experiment_path + model_name
with open(model_path, 'rb') as f:
    architecture = pickle.load(f)
    G = architecture['G_ema'].to(device)  # torch.nn.Module 
    D = architecture['D'].to(device)
os.chdir(current_wd)

### Assessor Model
This needs: 
- DinoV2 embeddings model
- Classifier model

In [6]:
from transformers import AutoImageProcessor, AutoModel


In [7]:
model_name = "facebook/dinov2-base"
device = set_device()
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model = model.to(device)

Using device: mps


In [ ]:
os.chdir(f"{ROOT_PATH}/4_Assessor/Category_Assessor/DinoV2")
from helper_classifier import ClassifierModel
classifier = torch.load(f"{DATA_PATH}/Models/Assessor/DinoV2/Classifier/dinov2_category_classifier.pt").to(device)

In [ ]:
classifier